# LLM-Modulo
- Adding verifiers for more robust generation of plans

In [1]:
from taskgen import *

In [2]:
import os
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

In [4]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

# Self-critique / reflection
- Does not work too well if LLM does not have the right abstraction space

In [13]:
res = strict_json(system_prompt = 'Count the number of r in strawberry',
            user_prompt = '',
            output_format = {'Explanation': 'type: str',
                             'Number of r': 'type: int'},
            llm = llm)

In [14]:
res

{'Explanation': 'The word "strawberry" contains the letter "r" twice.',
 'Number of r': 2}

In [15]:
res2 = strict_json(system_prompt = 'Critique the solution to "Count the number of r in strawberry"',
            user_prompt = res,
            output_format = {'Critique': 'type: str',
                             'Modified answer': 'type: str',
                             'Number of r': 'type: int'},
            llm = llm)

In [16]:
res2

{'Critique': 'The explanation is correct, but it could be more concise and directly answer the question without additional wording.',
 'Modified answer': 'The word "strawberry" contains the letter "r" 2 times.',
 'Number of r': 2}

# Critic to count letters

In [37]:
res = strict_json(system_prompt = 'Count the number of r in strawberry.',
            user_prompt = '',
            output_format = {'Explanation': 'type: str',
                             'Number of r': 'type: int'},
            llm = llm)

In [38]:
res

{'Explanation': 'The word "strawberry" contains the letter "r" twice.',
 'Number of r': 2}

In [63]:
count = "strawberry".count('r')

In [64]:
res2 = strict_json(system_prompt = f'Critique the solution to "Count the number of r in strawberry". Count of r from code: {count}',
            user_prompt = res,
            output_format = {'Critique': 'type: str',
                             'Modified answer': 'type: str',
                             'Number of r': 'type: int'},
            llm = llm)

In [65]:
res2

{'Critique': 'The count of r is incorrect; the letter "r" appears three times in "strawberry".',
 'Modified answer': 'The word "strawberry" contains the letter "r" three times.',
 'Number of r': 3}

# Tool use

In [22]:
def count_letters(word: str, letter: str) -> str:
    '''Counts letter in the word'''
    return word.count(letter)

In [23]:
count_letters('strawberry', 'r')

3

In [24]:
agent = Agent('Generalist Agent', 'Does anything', llm = llm).assign_functions(count_letters)

In [25]:
agent.run('Count the number of r in strawberry')

Observation: No subtasks have been completed yet for the task of counting the number of r in strawberry.
Thoughts: To complete the task, I need to count the occurrences of the letter "r" in the word "strawberry". This can be done using the count_letters function.
Subtask identified: Count the occurrences of the letter "r" in the word "strawberry" using the count_letters function.
Calling function count_letters with parameters {'letter': 'r', 'word': 'strawberry'}
> {'output_1': 3}

Observation: The task to count the number of occurrences of the letter "r" in the word "strawberry" has been completed, resulting in a count of 3.
Thoughts: Since the counting task is complete and the result is available, the next step is to finalize the task and present the output to the user.
Subtask identified: End Task
Task completed successfully!



[{'output_1': 3}]

In [26]:
agent.reply_user()

The assigned task was to count the number of occurrences of the letter "r" in the word "strawberry". According to the completed subtask, the output indicates that there are 3 instances of the letter "r" in the word "strawberry". Therefore, the final count of the letter "r" in "strawberry" is 3.


'The assigned task was to count the number of occurrences of the letter "r" in the word "strawberry". According to the completed subtask, the output indicates that there are 3 instances of the letter "r" in the word "strawberry". Therefore, the final count of the letter "r" in "strawberry" is 3.'